### jsonl -> csv

In [3]:
import json
import csv

# Specify the input and output file paths
input_file = 'NIKL_AU_2023_COMPETITION_v1.0/nikluge-au-2022-test.jsonl'
output_file = 'NIKL_AU_2023_COMPETITION_v1.0/test.csv'

data = []

# Read the JSONL file
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        # Remove any leading/trailing whitespace
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        # Parse the JSON object
        record = json.loads(line)
        data.append(record)

# Write data to a CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV column headers
    fieldnames = ['id', 'input', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write each JSON object as a row in the CSV file
    for record in data:
        writer.writerow(record)


### csv 를 padnas 로 읽어오기

In [1]:
# 데이터 읽어와보기
import pandas as pd
train_df = pd.read_csv("NIKL_AU_2023_COMPETITION_v1.0/train.csv")

In [3]:
train_df.head(3)

,id,input,output
0,nikluge-au-2022-train-000001,보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사...,1
1,nikluge-au-2022-train-000002,왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프...,1
2,nikluge-au-2022-train-000003,이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름...,1


### input 데이터에서 비식별화 토큰 찾기

In [31]:
# "&" 포함하는 것들
len(train_df[train_df['input'].str.contains('&')]['input']) # 1783개
train_df[train_df['input'].str.contains('&')]['input'].iloc[11]

'외교 경제 국방등 망쳤으면 조용히 &location&대통령의 간첩처분 받아라!'

- 하나씩 1783 개 다보긴 싫으니까 저 3가지 외에 다른게 있는 경우로 보자!

In [76]:
filtered_train_df = train_df[train_df['input'].str.contains('&')]['input']
refiltered_train_df = filtered_train_df[filtered_train_df.apply(lambda x: all(word not in x for word in ('&location&','&affiliation&','&name&','&company&','&brand&','&art&','&other&','&nama&','&affifiation&','&online-account&','&name','&compnay&','&anme&','& name&','&address&','&tel-num&','&naem&')))]
len(refiltered_train_df)

2

> 정리중 
- &location& = 지역/지명, &affiliation& = 단체 , &name& = 이름 , &company& = 회사 , &brand& = 브랜드
- &art& = 작품이름 , &online-account& = 플랫폼?, &address& = 주소, '&tel-num&' = 핸드폰 번호, &other& = 분류 외

- 휴먼에러?? : &affifiation& , &nama& , &name , &compnay& , &anme&  , & name& , &naem&

In [78]:
refiltered_train_df.iloc[1]

'어린여캐&늙은남캐 로맨틱한 연성도 많았고 ㄱ 애기들한테 요망한 로리라고 매도하는것도 싫었고'

In [60]:
type(train_df.output.values)

numpy.ndarray

### 라벨별 분포

In [16]:
len(train_df[train_df.output==0])

9794

In [4]:
len(train_df[train_df.output==1])

6786

### 학습 데이터 살펴보기

In [15]:
train_df[train_df.output==1].loc[34]

id                 nikluge-au-2022-train-000035
input     왕권시대에도 볼수 없던 독재를 &name&정권은 거침 없이 자행 !
output                                        1
Name: 34, dtype: object

### 정규표현식으로 비식별화 토큰 (&name&) 같은 거 검색해보기 그리고 special token 처리하기

- &name& -> "&name&"  -> [32001]
- &name& -> & , name , & -> [35, 767, 35]
- special token 에 "&name&" 

In [26]:
for i, j in train_df.items():
    print(i)
    print("0"*100)
    print(j)

id
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0        nikluge-au-2022-train-000001
1        nikluge-au-2022-train-000002
2        nikluge-au-2022-train-000003
3        nikluge-au-2022-train-000004
4        nikluge-au-2022-train-000005
                     ...             
16575    nikluge-au-2022-train-016576
16576    nikluge-au-2022-train-016577
16577    nikluge-au-2022-train-016578
16578    nikluge-au-2022-train-016579
16579    nikluge-au-2022-train-016580
Name: id, Length: 16580, dtype: object
input
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0        보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사...
1        왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프...
2        이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름...
3                                             인간의 탈을 쓰고...
4                              인기글에 짱깨뭐라하니까 댓글로 ㅂㄷㅂㄷ하네요...
                               ...   

### 토크나이저로 한번에 여러개의 문장 배치단위로 토크나이징
- 이때 input 은 list 형태로 넣어주셔야 에러가 안납니다.

In [12]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [33]:
for i,j in train_df.items():
    print(type(i[0]))
    print(type(j[0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>


### 스페셜 토큰으로 추가해보기
- docs : https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.add_special_tokens

In [79]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

/Users/jo_nyuk/.pyenv/versions/3.11.8/envs/Korean_AU/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [81]:
special_tokens = ['&location&','&affiliation&','&name&','&company&','&brand&','&art&','&other&','&nama&','&affifiation&','&online-account&','&name','&compnay&','&anme&','& name&','&address&','&tel-num&','&naem&']
tokenizer.add_special_tokens({'additional_special_tokens':special_tokens})

17

In [111]:
# 추가된 토큰 확인
print("추가된 스페셜 토큰:", tokenizer.special_tokens_map)

추가된 스페셜 토큰: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['&location&', '&affiliation&', '&name&', '&company&', '&brand&', '&art&', '&other&', '&nama&', '&affifiation&', '&online-account&', '&name', '&compnay&', '&anme&', '& name&', '&address&', '&tel-num&', '&naem&']}


In [110]:
input_text = "location&"  # location&"
print(tokenizer.encode(input_text))
print(tokenizer.decode(tokenizer.encode(input_text)))
for token in tokenizer.encode(input_text):
    print("-"*30)
    print(tokenizer.decode(token))

[2, 79, 6624, 5148, 10, 3]
[CLS] location & [SEP]
------------------------------
[CLS]
------------------------------
l
------------------------------
##oc
------------------------------
##ation
------------------------------
&
------------------------------
[SEP]
